In [1]:
import numpy as np
import pandas as pd

### Data Acquisition

- Data obtained from NASA's Prediction of Worldwide Energy Resource (POWER) [website](https://power.larc.nasa.gov/data-access-viewer/)
- Used GUI to generate a link for CSV data
  - https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,PS,WS50M,WD50M&community=RE&longitude=-85.9654&latitude=44.6385&start=20110401&end=20210331&format=CSV
  - First 12 lines are metadata
- Choose a point in Michigan, US as a hypothetical location for a community wind site

In [2]:
link = "https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,PS,WS50M,WD50M&community=RE&longitude=-85.9654&latitude=44.6385&start=20110401&end=20210331&format=CSV"
#read the data
df = pd.read_csv(link,skiprows=12)

In [3]:
df.head()

,YEAR,MO,DY,HR,T2M,PS,WS50M,WD50M
0,2011,3,31,18,0.79,98.38,1.86,336.01
1,2011,3,31,19,0.84,98.39,1.12,348.38
2,2011,3,31,20,1.04,98.36,0.29,7.70
3,2011,3,31,21,1.16,98.31,0.57,182.35
4,2011,3,31,22,1.22,98.27,1.28,193.10


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87672 entries, 0 to 87671
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   YEAR    87672 non-null  int64  
 1   MO      87672 non-null  int64  
 2   DY      87672 non-null  int64  
 3   HR      87672 non-null  int64  
 4   T2M     87672 non-null  float64
 5   PS      87672 non-null  float64
 6   WS50M   87672 non-null  float64
 7   WD50M   87672 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 5.4 MB


In [5]:
df.loc[:,'YEAR':'HR']

,YEAR,MO,DY,HR
0,2011,3,31,18
1,2011,3,31,19
2,2011,3,31,20
3,2011,3,31,21
4,2011,3,31,22
...,...,...,...,...
87667,2021,3,31,13
87668,2021,3,31,14
87669,2021,3,31,15
87670,2021,3,31,16


In [6]:
#Make new datetime column
df['dt'] = df.YEAR.astype(str) + '-' + df.MO.astype(str).str.pad(2,side='left',fillchar='0') + '-' + df.DY.astype(str).str.pad(2,side='left',fillchar='0') + ' ' + df.HR.astype(str).str.pad(2,side='left',fillchar='0')

In [7]:
#double check everything is padded appropriately
df.dt.head(10)

0    2011-03-31 18
1    2011-03-31 19
2    2011-03-31 20
3    2011-03-31 21
4    2011-03-31 22
5    2011-03-31 23
6    2011-04-01 00
7    2011-04-01 01
8    2011-04-01 02
9    2011-04-01 03
Name: dt, dtype: object

In [8]:
#convert to datetime
df.dt = pd.to_datetime(df.dt,format='%Y-%m-%d %H')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87672 entries, 0 to 87671
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   YEAR    87672 non-null  int64         
 1   MO      87672 non-null  int64         
 2   DY      87672 non-null  int64         
 3   HR      87672 non-null  int64         
 4   T2M     87672 non-null  float64       
 5   PS      87672 non-null  float64       
 6   WS50M   87672 non-null  float64       
 7   WD50M   87672 non-null  float64       
 8   dt      87672 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4)
memory usage: 6.0 MB


In [10]:
#set and sort the index
df.set_index('dt').sort_index()

,YEAR,MO,DY,HR,T2M,PS,WS50M,WD50M
dt,,,,,,,,
2011-03-31 18:00:00,2011,3,31,18,0.79,98.38,1.86,336.01
2011-03-31 19:00:00,2011,3,31,19,0.84,98.39,1.12,348.38
2011-03-31 20:00:00,2011,3,31,20,1.04,98.36,0.29,7.70
2011-03-31 21:00:00,2011,3,31,21,1.16,98.31,0.57,182.35
2011-03-31 22:00:00,2011,3,31,22,1.22,98.27,1.28,193.10
...,...,...,...,...,...,...,...,...
2021-03-31 13:00:00,2021,3,31,13,0.81,99.61,6.53,316.21
2021-03-31 14:00:00,2021,3,31,14,0.73,99.67,6.08,323.00
2021-03-31 15:00:00,2021,3,31,15,0.54,99.74,5.96,331.42


In [11]:
#Drop this all into a function
def getMIWindData():
    '''
    Gets wind related atmospheric data from NASA's POWER website for a location in small town Michigan
    
    Returns: Pandas Dataframe with datetime index
    '''
    #set link for 
    link = "https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,PS,WS50M,WD50M&community=RE&longitude=-85.9654&latitude=44.6385&start=20110401&end=20210331&format=CSV"
    #return dataframe
    return getHourlyPOWERData(link)

def getHourlyPOWERData(link):
    '''
    Given a CSV-type URL for NASA's POWER website, returns a formatted dataframe with a datetime index.
    Assumes hourly data is requested. 
    
    Returns: Pandas DataFrame w/ Datetime index 
    Parameters: 
        (R) link: URL dynamically generated by NASA's POWER interface. Use CSV setting in GUI
        
    IMPROVEMENTS:
    So this is a bit ugly as it assumes 12 header rows (which is probably only the case when 4 parameters are chosen).
    - Create function to get last line of header
    - - For NASA site, that is with "-END HEADER-", however ideally I will functionize it to:
    - - - handle any string to indicate last line of metadata
    - - - handle any string to indicate first line of actual data
    - - - handle any string to indicate prefix of metadata
    - This will also mean we won't take a link, but rather will download the data, then open the file to find the header
    - ALT: mayb be able to calculate rows to skip via parameter count in link (Still assumes standard # of metadata lines)
    '''
    #get last row of metadata
    meta_end = 12
    #read the data
    df = pd.read_csv(link,skiprows=meta_end)
    
    #Make new datetime column
    df['dt'] = df.YEAR.astype(str) + '-' + df.MO.astype(str).str.pad(2,side='left',fillchar='0') + '-' + df.DY.astype(str).str.pad(2,side='left',fillchar='0') + ' ' + df.HR.astype(str).str.pad(2,side='left',fillchar='0')
    #Convert column to datetime
    df.dt = pd.to_datetime(df.dt,format='%Y-%m-%d %H')
    
    #set index, sort and return dt
    return df.set_index('dt').sort_index()

##### test those functions

In [12]:
import utils

In [13]:
df2 = utils.getMIWindData()

In [14]:
df2.info() 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 87672 entries, 2011-03-31 18:00:00 to 2021-03-31 17:00:00
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   YEAR    87672 non-null  int64  
 1   MO      87672 non-null  int64  
 2   DY      87672 non-null  int64  
 3   HR      87672 non-null  int64  
 4   T2M     87672 non-null  float64
 5   PS      87672 non-null  float64
 6   WS50M   87672 non-null  float64
 7   WD50M   87672 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 6.0 MB


In [15]:
df2.head()

,YEAR,MO,DY,HR,T2M,PS,WS50M,WD50M
dt,,,,,,,,
2011-03-31 18:00:00,2011,3,31,18,0.79,98.38,1.86,336.01
2011-03-31 19:00:00,2011,3,31,19,0.84,98.39,1.12,348.38
2011-03-31 20:00:00,2011,3,31,20,1.04,98.36,0.29,7.70
2011-03-31 21:00:00,2011,3,31,21,1.16,98.31,0.57,182.35
2011-03-31 22:00:00,2011,3,31,22,1.22,98.27,1.28,193.10
